## Create Dataframe of all Cities

#### Import packages

In [ ]:
import osmnx as ox
import numpy as np
import pandas as pd
import multiprocessing as mp
import pickle
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import ticker as tick
from datetime import datetime
import os

In [ ]:
city_list = pd.read_csv('SN/city_list.csv')
city_list = city_list.rename(columns={'0': 'City'})
print(city_list.head())

#### neu, da Grafken Mist.

In [ ]:
path = 'C:/Users/Besitzer/Desktop/Geospatial Analysis/Data'
city_list = []
bearing_list = []
bearing_w_list = []
for file in os.listdir(path):
    #print(file.replace('.pkl',''))
    data = pd.read_pickle(path + '/' + file)
    city_list.append(file.replace('.pkl', ''))
    bearing_list.append(data['bearing'])
    bearing_w_list.append(data['bearing_weighted'])
    
bears = []
bears_w = []
for j in range(len(bearing_list)):
    bins = []
    bins_w = []
    for i in range(36):
        if i == 0:
            bins.append(len([ele for ele in bearing_list[j] if (ele > 355) or (ele <= 5)])) #no relative values!
            bins_w.append(len([ele for ele in bearing_w_list[j] if (ele > 355) or (ele <= 5)]))
        else: #i = 1 -->  10*i - 5 , 10*i + 5
            bins.append(len([ele for ele in bearing_list[j] if (ele > 10*i-5) and (ele <= 10*i+5)]))
            bins_w.append(len([ele for ele in bearing_w_list[j] if (ele > 10*i-5) and (ele <= 10*i+5)]))
    bears.append(bins)
    bears_w.append(bins_w)    

In [ ]:
#bears[0]: [3951,127,265,75,126,112,138,80,125,4032,143,284,120,156,151,92,107,152...]
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

for i in range(len(city_list)):
    #unweighted
    bins_number = 36  # the [0, 360) interval will be subdivided into this
    # number of equal bins
    #bins = np.linspace(0.0, 2 * np.pi, bins_number + 1)
    bins = np.linspace(0, 2 * np.pi , bins_number + 1)

    #angles = 2 * np.pi * np.random.rand(n_numbers)
    #n, _, _ = plt.hist(bearings[city_list['City'][i]], bins_number, density = True)
    #density:
    n = [ele/sum(bears[i]) for ele in bears[i]]

    plt.clf()
    width = 2 * np.pi / bins_number
    ax = plt.subplot(1, 1, 1, projection='polar')
    bars = ax.bar(bins[:bins_number], n, width=width, bottom=0.0, color='steelblue', edgecolor='black', linewidth=0.4)
    #plt.yticks([])
    #ax.set_yticks([])
    ax.set_xticklabels(['N', '', 'E', '', 'S', '', 'W', ''])
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.tick_params(axis='y', colors='silver')
    #ax.set_axisbelow(True)
    ax.yaxis.set_major_locator(MaxNLocator(4))
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(8) 
    pos=ax.get_rlabel_position()
    ax.set_rlabel_position(pos+10)
    plt.title(city_list[i])
    for bar in bars:
        bar.set_alpha(1)
    plt.grid(True,linestyle='--', linewidth=0.4)
    plt.savefig('Plots/bear_' + city_list[i] + '.pdf')
    plt.show()
    
    n = [ele/sum(bears_w[i]) for ele in bears_w[i]]

    plt.clf()
    width = 2 * np.pi / bins_number
    ax = plt.subplot(1, 1, 1, projection='polar')
    bars = ax.bar(bins[:bins_number], n, width=width, bottom=0.0, color='steelblue', edgecolor='black', linewidth=0.4)
    #plt.yticks([])
    #ax.set_yticks([])
    ax.set_xticklabels(['N', '', 'E', '', 'S', '', 'W', ''])
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.tick_params(axis='y', colors='silver')
    #ax.set_axisbelow(True)
    ax.yaxis.set_major_locator(MaxNLocator(4))
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(8) 
    pos=ax.get_rlabel_position()
    ax.set_rlabel_position(pos+10)
    plt.title(city_list[i])
    for bar in bars:
        bar.set_alpha(1)
    plt.grid(True,linestyle='--', linewidth=0.4)
    plt.savefig('Plots/bear_w_' + city_list[i] + '.pdf')
    plt.show()

#### alt, doofe Grafiken

In [ ]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

for i in range(50, len(city_list)+1):
    print(city_list['City'][i], datetime.now())
    G = pickle.load( open( "SN/" + 'network_graph_POI_' + city_list['City'][i] + ".p", "rb" ) )
    
    # what sized area does our network cover in square meters?
    G_proj = ox.project_graph(G)
    nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
    graph_area_m = nodes_proj.unary_union.convex_hull.area
    
    # calculate basic and extended network stats, merge them together, and display
    stats = ox.basic_stats(G, area=graph_area_m)
    extended_stats = ox.extended_stats(G, ecc=True, bc=True, cc=True)
    for key, value in extended_stats.items():
        stats[key] = value
    stats = pd.Series(stats)
    stats['oneway'] = 0
    stats['twoway'] = 0
    for u, v, key, data in G.edges(keys=True, data=True):
        if data['oneway']:
            stats['oneway'] += 1
        else:
            stats['twoway'] += 1
     
    
    #################################
    weight_by_length = False
    bearings = {}
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[city_list['City'][i]] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[city_list['City'][i]] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    stats['bearing'] = list(bearings[city_list['City'][i]])

    #n_numbers = 100
    bins_number = 36  # the [0, 360) interval will be subdivided into this
    # number of equal bins
    #bins = np.linspace(0.0, 2 * np.pi, bins_number + 1)
    bins = np.linspace(+np.pi/18, 2 * np.pi + np.pi/18, bins_number + 1)

    #angles = 2 * np.pi * np.random.rand(n_numbers)
    n, _, _ = plt.hist(bearings[city_list['City'][i]], bins_number, density = True)
    plt.clf()
    width = 2 * np.pi / bins_number
    ax = plt.subplot(1, 1, 1, projection='polar')
    bars = ax.bar(bins[:bins_number], n*10, width=width, bottom=0.0, color='steelblue', edgecolor='black', linewidth=0.4)
    #plt.yticks([])
    #ax.set_yticks([])
    ax.set_xticklabels(['N', '', 'E', '', 'S', '', 'W', ''])
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.tick_params(axis='y', colors='silver')
    ax.yaxis.set_major_locator(MaxNLocator(4))
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(8) 
    pos=ax.get_rlabel_position()
    ax.set_rlabel_position(pos+10)
    plt.title(city_list['City'][i])
    for bar in bars:
        bar.set_alpha(1)
    plt.savefig('Plots/bearings_' + city_list['City'][i] + '.pdf')
    plt.show()
    ############################################
    weight_by_length = True
    bearings = {}
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[city_list['City'][i]] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[city_list['City'][i]] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    stats['bearing_weighted'] = list(bearings[city_list['City'][i]])

    #n_numbers = 100
    bins_number = 36  # the [0, 360) interval will be subdivided into this
    # number of equal bins
    #bins = np.linspace(0.0, 2 * np.pi, bins_number + 1)
    bins = np.linspace(+np.pi/18, 2 * np.pi + np.pi/18, bins_number + 1)

    #angles = 2 * np.pi * np.random.rand(n_numbers)
    n, _, _ = plt.hist(bearings[city_list['City'][i]], bins_number, density = True)
    plt.clf()
    width = 2 * np.pi / bins_number
    ax = plt.subplot(1, 1, 1, projection='polar')
    bars = ax.bar(bins[:bins_number], n*10, width=width, bottom=0.0, color='steelblue', edgecolor='black', linewidth=0.4)
    #plt.yticks([])
    #ax.set_yticks([])
    ax.set_xticklabels(['N', '', 'E', '', 'S', '', 'W', ''])
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.tick_params(axis='y', colors='silver')
    ax.yaxis.set_major_locator(MaxNLocator(4))
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(8) 
    pos=ax.get_rlabel_position()
    ax.set_rlabel_position(pos+10)
    plt.title(city_list['City'][i])
    for bar in bars:
        bar.set_alpha(1)
    plt.savefig('Plots/bearings_weighted_' + city_list['City'][i] + '.pdf')
    plt.show()
    
    ############################################
    stats.to_pickle('Data/' + city_list['City'][i] + '.pkl')